<img src="img/viu_logo.png" width="200"><img src="img/python_logo.png" width="250"> *Mario Cervera*

# Introducción a la Programación - Actividad Final

El fichero *grafo.txt* define un grafo dirigido ponderado. Cada fila del fichero contiene tres items separados entre sí por un espacio. Estos tres items definen una arista y su peso. Por ejemplo, la fila "a b 2" define una arista *(a, b)*, cuyo peso es 2, y donde *a* y *b* son nodos del grafo. La arista tiene *a* como origen y *b* como destino.

1.1. Crea una clase *Arista* que represente una arista del grafo, con su nodo origen, su nodo destino y su peso. La clase debe sobreescribir el operador que permite que las instancias de la clase puedan representarse apropiadamente en formato *string*. Añade documentación a la clase.

In [1]:
# 1.1

class Arista:
    def __init__(self, origen, destino, peso):
        self.origen = origen
        self.destino = destino
        self.peso = peso
        
    def __str__(self):
        return f'({self.origen}, {self.destino}, {self.peso})'

1.2. Crea una clase abstracta *Grafo* que represente un grafo, pero sin proporcionar detalles sobre su representación en memoria. Esta clase abstracta contendrá un constructor que recibirá un parámetro: la ruta a un fichero de texto, de donde la clase *Grafo* podrá extraer la definición del grafo. La clase, al ser abstracta, no puede crear el grafo, pero sí puede procesar el fichero y usar un método abstracto *anyadir_arista*. Añadir también a la clase *Grafo* otro método abstracto *contiene_arista* que permita comprobar la presencia de una arista en el grafo. Ambos métodos recibirán una *Arista* como parámetro. Añade documentación a la clase.

In [119]:
# 1.2

from abc import ABC
from abc import abstractmethod

class Grafo(ABC):
    def __init__(self, direccion_fichero):
        self.procesar_fichero(open(direccion_fichero))
        
    def procesar_fichero(self, direccion_fichero):    
        with direccion_fichero as fichero:
            lineas = fichero.readlines()
            for linea in lineas: 
                parte_de_arista = linea.split()
                arista = Arista(parte_de_arista[0], parte_de_arista[1], parte_de_arista[2])
                self.anyadir_arista(arista)  
                
    @abstractmethod
    def anyadir_arista(self, arista):
        assert type(arista) == Arista, "El objeto que desea anyadir no es de tipo arista"
        pass
    
    @abstractmethod
    def contiene_arista(self, arista):
        assert type(arista) == Arista, "El objeto que desea buscar en el grafo no es de tipo arista"
        pass

1.3. Crea una subclase *GrafoListasAdyacencia*. Esta clase implementará el método *anyadir_arista* de manera que se creen las listas de adyacencia de manera apropiada. La clase deberá también implementar el método *contiene_arista*. Añade documentación a la clase.

Nota: observad que en las listas de adyacencia no debéis almacenar objetos de tipo *Arista*, ya que esto crearía duplicación innecesaria de información en memoria.

In [120]:
# 1.3

class GrafoListasAdyacencia(Grafo):
    def __init__(self, direccion_fichero):
        self.lista_adyacencia = {}
        super().__init__(direccion_fichero)
        
        
    def anyadir_arista(self, arista):
        super().anyadir_arista(arista)
        if arista.origen in self.lista_adyacencia:
            self.lista_adyacencia[arista.origen].insert(0, (arista.destino, arista.peso))
        else:
            self.lista_adyacencia[arista.origen] = [(arista.destino, arista.peso)]
                
    def contiene_arista(self, arista):
        lista_adyacencia = self.lista_adyacencia
        if arista.origen in lista_adyacencia:
            for destino_y_peso in lista_adyacencia[arista.origen]:
                if destino_y_peso == (arista.destino, arista.peso):
                    return True
        return False

grafo_lista = GrafoListasAdyacencia("res/grafo.txt")

print(grafo_lista.lista_adyacencia)

{'a': [('c', '3'), ('b', '1')], 'b': [('e', '3')], 'c': [('d', '1'), ('a', '2')], 'd': [('f', '1'), ('e', '2'), ('a', '1')], 'e': [('f', '4'), ('c', '3')], 'f': [('g', '1')], 'g': [('b', '2')]}


1.4. Crea una subclase *GrafoMatrizAdyacencia*. Esta clase implementará el método *anyadir_arista* de manera que se cree la matriz de adyacencia de manera apropiada. Una matriz de adyacencia es una matriz cuadrada que indica, para cada par de nodos, si son adyacentes o no. Más formalmente, dado un grafo con nodos *U = { u<sub>1</sub>, u<sub>2</sub>, ..., u<sub>n</sub> }*, la matriz de adyacencia es una matriz *n x n* donde un elemento *A<sub>ij</sub>* de la matriz es *X* cuando el grafo posee una arista del nodo *u<sub>i</sub>* al nodo *u<sub>j</sub>* con peso *X*, y 0 cuando no existe tal arista o tiene peso 0.

Nota: para este ejercicio, podéis asumir que se sabe de antemano (es decir, antes de procesar el fichero) que el grafo tiene 7 nodos: 'a', 'b', 'c', 'd', 'e', 'f' y 'g'.

In [110]:
#1.4

class GrafoMatrizAdyacencia(Grafo):
    def __init__(self, direccion_fichero):
        self.matriz_adyacencia = {}
        super().__init__(direccion_fichero)
        
    def anyadir_arista(self, arista):
        super().anyadir_arista(arista)
        posicion_destino = ord(arista.destino) - 97                      
        if arista.origen not in self.matriz_adyacencia:
            self.matriz_adyacencia[arista.origen] = [0 for pocision in range(7)]
        self.matriz_adyacencia[arista.origen][posicion_destino] = int(arista.peso)
    
    def contiene_arista(self, arista):
        super().contiene_arista(arista)
        posicion_destino = ord(arista.destino) - 97
        if arista.origen in self.matriz_adyacencia and self.matriz_adyacencia[arista.origen][posicion_destino] == int(arista.peso):
            return True 
        return False
            
grafo_matriz= GrafoMatrizAdyacencia("res/grafo.txt")

print(grafo_matriz.matriz_adyacencia)

{'a': [0, 1, 3, 0, 0, 0, 0], 'b': [0, 0, 0, 0, 3, 0, 0], 'c': [2, 0, 0, 1, 0, 0, 0], 'd': [1, 0, 0, 0, 2, 1, 0], 'e': [0, 0, 3, 0, 0, 4, 0], 'f': [0, 0, 0, 0, 0, 0, 1], 'g': [0, 2, 0, 0, 0, 0, 0]}


1.5. Crea una función que, dado un grafo y una arista, compruebe si la arista existe en el grafo y muestre un mensaje apropiado por pantalla en cualquier caso. Utiliza esta función para comprobar la existencia/ausencia de varias aristas en una instancia de un grafo basado en listas de adyacencia y también en un grafo basado en matriz de adyacencia. El resultado debería ser el mismo en ambos casos, ya que la existencia o ausencia de una arista en un grafo no depende de cómo el grafo está representado internamente.

In [102]:
#1.5 

def comprobar_arista_en_grafo(grafo, arista):
    assert type(arista) == Arista, "El segundo parametro de la funcion no es una arista"
    if type(grafo) is GrafoMatrizAdyacencia or type(grafo) is GrafoListasAdyacencia:
        if grafo.contiene_arista(arista) == True:
            print(f"La arista {arista} existe dentro del grafo")
        else:
            print(f"La arista {arista} no existe dentro del grafo")  
    else:
        raise TypeError("El primer parametro de la funcion no es un grafo")
    
    
arista1 = Arista('a', 'b', '1')
arista2 = Arista('c', 'd', '1')
arista3 = Arista('h', 'c', '4')
arista4 = Arista('f', 'g', '6')

#comprobar_arista_en_grafo('a', arista1)
comprobar_arista_en_grafo(grafo_lista, arista1)
comprobar_arista_en_grafo(grafo_lista, arista2)
comprobar_arista_en_grafo(grafo_lista, arista3)
comprobar_arista_en_grafo(grafo_lista, arista4)

comprobar_arista_en_grafo(grafo_matriz, arista1)
comprobar_arista_en_grafo(grafo_matriz, arista2)
comprobar_arista_en_grafo(grafo_matriz, arista3)
comprobar_arista_en_grafo(grafo_matriz, arista4)


La arista (a, b, 1) existe dentro del grafo
La arista (c, d, 1) existe dentro del grafo
La arista (h, c, 4) no existe dentro del grafo
La arista (f, g, 6) no existe dentro del grafo
La arista (a, b, 1) existe dentro del grafo
La arista (c, d, 1) existe dentro del grafo
La arista (h, c, 4) no existe dentro del grafo
La arista (f, g, 6) no existe dentro del grafo
